In [ ]:
import numpy as np 
import pandas as pd 

import warnings
warnings.filterwarnings("ignore")

# plan
## About dataset

This dataset is a simulation of a large online store and contains data on:
Order details
Customers
Shipping
Products
Sales, discounts, and profits.

## customer Tasks
- A clean dataset (remove duplicates, handle missing values).
- Sales by Region and Category
- Profit by Sub-Category
- Top 10 Customers by Sales
- Monthly Sales Trend
- Some business insights about which products or categories perform best and where we are losing money.

In [ ]:
path=  "/kaggle/input/superstore-dataset-final/Sample - Superstore.csv"
df = pd.read_csv(path, encoding="latin1")
df.head()

In [ ]:
print("Before drop duplocate:", df.shape)
df = df.drop_duplicates()
print("After drop duplocate:", df.shape)
print(df.isna().sum())

In [ ]:
print(df.shape)
#print(df.info())
df.describe().T

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# الأعمدة اللي هنشتغل عليها
cols = ["Sales", "Quantity", "Discount", "Profit"]

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i, col in enumerate(cols):
    # Boxplot
    sns.boxplot(y=df[col], ax=axes[1, i])
    axes[1, i].set_title(f"Boxplot of {col}")
    
     # Histogram
    sns.histplot(df[col], bins=30, kde=True, ax=axes[0, i])
    axes[0, i].set_title(f"Histogram of {col}")
    

plt.tight_layout()
plt.show()

### 1. Basic Stats

- Sales: What is the average sales volume? = 229.8  What is the maximum sales volume? = 22,638


- Quantity: Most orders are sold in small quantities. Average quantity sold ≈ 3.8 units.Most orders are small (between 2–5 units), and the maximum order was 14 units.

- Discount: Discounts of up to 0.8% (80%) can affect profit.

- Profit: Are there products that are losing profit (negative profit)?

In [ ]:
cols = ["Sales", "Quantity", "Discount", "Profit"]

for i in cols:
    s= df[i].value_counts().sort_index()
    print(s.to_frame())

In [ ]:
df.columns

# ✅ 2. Sales by Region and Category

In [ ]:
df.Category.value_counts().plot(kind="pie")
plt.title('Sales by Category')

In [ ]:
sales_region_category = df.groupby(['Region', 'Category'])['Sales'].sum().reset_index()

plt.figure(figsize=(10,6))
sns.barplot(data=sales_region_category, x='Region', y='Sales', hue='Category')
plt.title('Sales by Region and Category')
plt.show()


- Sales are at their lowest in the south.
- Sales are at their highest in the east and west.
- Technology is the most valuable in all regions, especially the eastern region.

# ✅ 3. Profit by Sub-Category

In [ ]:
profit_subcat = df.groupby("Sub-Category")["Profit"].sum().reset_index().sort_values(ascending=False,by="Profit")

plt.figure(figsize=(12,6))
sns.barplot(data=profit_subcat, x='Sub-Category', y='Profit' , 
           palette = sns.color_palette ("viridis" , n_colors=len(profit_subcat) ))
           
plt.xticks(rotation=45)
plt.title('Profit by Sub-Category')
plt.show()

# ✅ 4. Top 10 Customers by Sales

In [ ]:
df.columns

In [ ]:
# "Customer Name" , "Sales"
Customer_by_sales = df.groupby("Customer Name")["Sales"].sum().reset_index()
Top_10_Customers= Customer_by_sales.sort_values(by="Sales", ascending= False).head(10)

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(data=Top_10_Customers, x='Customer Name', y='Sales' ,
            palette=sns.color_palette("viridis", n_colors=len(Top_10_Customers)))
            
plt.xticks(rotation=45)
plt.title('Top 10 Customers by Sales')
plt.show()

# ✅ 5. Monthly Sales Trend

In [ ]:
# نتأكد إن عمود Order Date متحول لتاريخ
df['Order Date'] = pd.to_datetime(df['Order Date'])

# نضيف عمود شهر-سنة
df['Month'] = df['Order Date'].dt.to_period('M').astype(str)

# نجمع المبيعات حسب الشهر
monthly_sales = df.groupby('Month')['Sales'].sum().reset_index()

# نتأكد إن Sales أرقام
monthly_sales['Sales'] = monthly_sales['Sales'].astype(float)

# الرسم
plt.figure(figsize=(12,6))
sns.lineplot(data=monthly_sales, x='Month', y='Sales', marker='o')
plt.title('Monthly Sales Trend')
plt.xticks(rotation=90)
plt.show()


In [ ]:
df["Order Date"] = pd.to_datetime(df["Order Date"], errors="coerce")
df["Month"] = df["Order Date"].dt.to_period("M").dt.to_timestamp()
df['Order Date'] = pd.to_datetime(df['Order Date'])

monthly_sales = (
    df.groupby("Month", as_index=False)["Sales"].sum()
    .sort_values("Month")
)

In [ ]:
path=  "/kaggle/input/superstore-dataset-final/Sample - Superstore.csv"
df = pd.read_csv(path, encoding="latin1")

In [ ]:
df["Order Date"]

In [ ]:
plt.figure(figsize=(14, 7))

# Lineplot مع تدريج ألوان
sns.lineplot(
    data=monthly_sales,
    x="Month", y="Sales",
    marker="*",
    linewidth=1,
    #color="#2E86AB"  
)

# تلوين النقاط individually
points = plt.scatter(
    monthly_sales["Month"],
    monthly_sales["Sales"],
    c=monthly_sales["Sales"],
    cmap="viridis", s=100, edgecolor="black", linewidth=0.5
)

plt.colorbar(points, label="Sales Value")

# عنوان وأوصاف
plt.title("Monthly Sales Trend", fontsize=18, fontweight="bold", color="#500")
plt.xlabel("Month", fontsize=14)
plt.ylabel("Total Sales", fontsize=14)


plt.xticks(rotation=90)

plt.tight_layout()
plt.show()


# Losses RCA

In [ ]:
df_loss= df[df["Profit"]<0]

In [ ]:
df_loss["Product ID"].value_counts().sort_values().tail(15).plot(kind="bar")

Top 15 losses with FUR -CH, TA  and OFF -BI, ST

In [ ]:
df_loss["Discount"].value_counts().sort_values().tail(5).plot(kind="bar")
# الخصم سبب رئيسي في الخسارة لكن العلاقة عير ثابتة - حيث يوجد خصم 20بالمئة فقط تسبب في 500 عملية خسارة

In [ ]:
df_loss_disc_20 = df_loss[df_loss["Discount"]==0.2]
df_loss_disc_20[["Category"	,"Sub-Category"]].value_counts().sort_values().tail(5)#.plot(kind="bar")

In [ ]:
df_loss_disc_20[["Category"]].value_counts().sort_values().tail(5)#.plot(kind="bar")